# RW - LM spec migration management v2 -> v4.

The aim of this script is to programatically allow us to move from the v2 layer spec from layer manager used in RW layers to the v4.

We will take as started point rw past migration and latest migration done by gfw. plus scripts develop for OW migration management from staging and productions


Tasks prior running the process:

* RW management functions
* Develop functions/code to move from spect v2 to v4
* Develop test to ensure functions works as expected

Finally the workflow will be excecuted one provider type at a time, and will be as follows:  

* Migrate to staging 5 layers of each provider
* For those 5 layers we will run the script to migrate and validate.
* Once Front is happy with the results we will trigger the full migration

In [1]:
### Run functions developed in the notebooks
%run './pydantic_data_classes_rw.ipynb'

In [2]:
%run './RW-API_management_functions.ipynb'

builtins==None
builtins==None
requests_toolbelt.sessions==None
types==None
json==2.0.9
logging==0.5.1.2
getpass==None


In [19]:
%run './RW-LM4_migration_functions.ipynb'

              ] \ reify:yarn: timing reify:loadBundles Completed in 0msK
changed 1 package, and audited 2 packages in 667ms

found 0 vulnerabilities
npm notice 
npm notice New major version of npm available! 7.7.6 -> 8.3.1
npm notice Changelog: https://github.com/npm/cli/releases/tag/v8.3.1
npm notice Run npm install -g npm@8.3.1 to update!
npm notice 


In [4]:
from IPython.display import JSON, display
from requests.exceptions import HTTPError

In [5]:
session = rwApiSession()

You are login into RW api
Email: alicia.arenzana@vizzuality.com
Password: ········
Successfully logged into RW api


### Explore and get some metrics about the layers currently in RW

In [6]:
layerList = getLayerList(session)

In [7]:
display(f' currently available {len(layerList)} layers in the api')

' currently available 5445 layers in the api'

In [10]:
display(layerList[0])
JSON(layerList[0].json(exclude_none=True))

**Layer**: [1993 Electricity Distribution Losses (kWh, billions)](https://api.resourcewatch.org/v1/dataset/a36bf77b-4d10-4855-8ef6-aece72e23971/layer/b9d1b315-efc3-433a-82ad-22e841c34f5a)

/opt/conda/lib/python3.8/site-packages/IPython/core/display.py:911: UserWarning: JSON expects JSONable dict or list, not JSON strings
  warnings.warn("JSON expects JSONable dict or list, not JSON strings")


<IPython.core.display.JSON object>

In [9]:
providers = {}
for layer in layerList:
    if layer.attributes.provider in providers.keys():
        providers[layer.attributes.provider]['count'] += 1
        providers[layer.attributes.provider]['ids'].append(layer.id)
    else:
        providers[layer.attributes.provider] = {'count': 1, 'ids':[layer.id]} 

for k,v in providers.items():
    print(f'{k}: {v["count"]}')

gee: 1601
cartodb: 3507
leaflet: 302
featureservice: 19
wms: 15
None: 1


This means in our migration we will need to only take care of 5 type of connectors

#### Review the layer with None as provider and exclude it from our migration

In [12]:
for layer in layerList:
    if layer.attributes.provider is None:
        display(layer)
        display(JSON(layer._url))

**Layer**: [Brazilian Amazon Deforestation Alerts](https://api.resourcewatch.org/v1/dataset/8870d62f-0240-4275-910a-361f0ef347e0/layer/0c07f80c-129d-47b3-9099-99036883a215)

<IPython.core.display.JSON object>

#### Lets have a look at the layerType that indicates a successfull migration from v1 to v2

In [13]:
layerTypes = {}
for layer in layerList:
    if layer.attributes.layerConfig.get('layerType') in layerTypes.keys():
        layerTypes[layer.attributes.layerConfig.get('layerType')] += 1
    else:
        layerTypes[layer.attributes.layerConfig.get('layerType')] = 1
layerTypes

{'vector': 2735, None: 1874, 'raster': 828}

#### From those that look unsusscessfull lets have a look at the providers, we will need to be extra carefull with this ones:

In [14]:
layer2Types = {}
for layer in layerList:
    if layer.attributes.layerConfig.get('layerType') is None:
        if layer.attributes.provider in layer2Types.keys():
            layer2Types[layer.attributes.provider] += 1
        else:
            layer2Types[layer.attributes.provider] = 1
layer2Types

{'gee': 1435,
 'cartodb': 282,
 'featureservice': 18,
 'wms': 15,
 'leaflet': 123,
 None: 1}

#### Add some testing layers from prod to staging from each different type so we can have some testing data.

In [15]:
testing_layers = []
try:
    for k,v in providers.items():
        print(f'{k}: ')
        for ID in v["ids"][:5]:
            layer = parse_obj_as(Layer, session.get(f'v1/layer/{ID}').json().get('data'))
            display(layer)
            copyLayer = layer.copy(deep=True)
            copyLayer.attributes.env = 'staging'
            testing_layers.append(session.post(f'v1/dataset/{copyLayer.attributes._datasetId}/layer', 
                                data = copyLayer.attributes.json(exclude_none=True))\
                                .json().get('data').get('id'))
except:
    pass

cartodb: 


**Layer**: [1993 Electricity Distribution Losses (kWh, billions)](https://api.resourcewatch.org/v1/dataset/a36bf77b-4d10-4855-8ef6-aece72e23971/layer/b9d1b315-efc3-433a-82ad-22e841c34f5a)

**Layer**: [1994 Electricity Distribution Losses (kWh, billions)](https://api.resourcewatch.org/v1/dataset/a36bf77b-4d10-4855-8ef6-aece72e23971/layer/1a2fb600-95f6-4aaa-b2cc-7d18c4510cd8)

**Layer**: [1995 Electricity Distribution Losses (kWh, billions)](https://api.resourcewatch.org/v1/dataset/a36bf77b-4d10-4855-8ef6-aece72e23971/layer/115ea29a-0dde-4dd8-8c1f-7084c90936f5)

**Layer**: [1996 Electricity Distribution Losses (kWh, billions)](https://api.resourcewatch.org/v1/dataset/a36bf77b-4d10-4855-8ef6-aece72e23971/layer/e7f1da28-0a28-43bd-a7eb-b56c664e7fc7)

...

**Layer**: [Projected Change (2041-2070) in Average Annual Precipitation Lower Emissions](https://api.resourcewatch.org/v1/dataset/ffe316c8-c059-4ed6-a5c9-bc640afe4d0e/layer/1e0931ef-39d3-4d30-ac56-923bf065ab45)

**Layer**: [Projected Average Annual Precipitation (2041_2070) Higher Emissions](https://api.resourcewatch.org/v1/dataset/90ff981d-200e-4010-be0e-b7b00768fa50/layer/4de30682-f19e-4d78-a568-5139ce38b376)

**Layer**: [Hurricane Tracks (Cat 3_5 Hurricanes Small Scale)](https://api.resourcewatch.org/v1/dataset/303bd52b-020f-4744-87c5-8da38fffd738/layer/a4c7856a-ac31-4b61-9e46-5be586bda386)

wms: 


**Layer**: [South America try 4](https://api.resourcewatch.org/v1/dataset/33156582-1d90-48d7-9db8-17330cae3afc/layer/db597954-9ec3-4b59-b60d-ac61e6e21ed2)

In [16]:
testing_layers

['46981bb8-ed82-4604-ad70-45e7b020c2de',
 '9ec93af0-c2fd-4f43-ab0d-b9b3e39ec1ef',
 '6086f11f-04fe-487d-8cc3-e35c318b17c1',
 '6984738a-d17a-4b44-b786-7bafec87be0f',
 '7e19037c-198b-4dbf-8c4d-acd1b609c4c6',
 '0490bdf9-15fa-4dbf-ae4a-0db4e8ea0e06',
 '17c64870-d3e6-45b6-a90a-c3ef106c91e7',
 '2023f875-5555-470e-8c68-b26b92d905aa',
 '4440b85d-af5a-4d0f-9177-a39a72bf79e1',
 '827e7618-db5d-4cd5-a5df-8d05fce4f574',
 '04296688-95fb-4d26-855d-bac090714789',
 'b7d4c7aa-ceca-4dc7-91bf-6d1e81d62d46',
 '0e96e261-87dd-47f0-b410-15f9edd3bc4f',
 '2a0112e7-a427-44a6-8184-56aece641595',
 '0cefc51b-91b3-461e-9204-1e5a58869ad6',
 '1db9bbc2-c546-4323-b21b-03947bb6f731',
 '5b82bcef-d4d6-433d-8df3-a64fb9ec113f',
 '05822741-9abf-41a3-a6dd-b912cfa62ac5',
 'fe8c4133-c529-48be-a8f4-2ab62daeabc5',
 'a3cd5963-ac93-4f33-932c-30af79b5f39a']

#### Start migration on the testing layers

In [18]:
createUpdatelayerSchemas(session, testing_layers)

  0%|          | 0/20 [00:00<?, ?it/s]

yarn install v1.22.17
[1/4] Resolving packages...
success Already up-to-date.
Done in 0.10s.


warning Skipping preferred cache folder "/home/jovyan/.cache/yarn" because it is not writable.
warning Selected the next writable cache folder in the list, will be "/tmp/.yarn-cache-1000".
  5%|▌         | 1/20 [00:02<00:38,  2.05s/it]

yarn install v1.22.17
[1/4] Resolving packages...
success Already up-to-date.
Done in 0.09s.


warning Skipping preferred cache folder "/home/jovyan/.cache/yarn" because it is not writable.
warning Selected the next writable cache folder in the list, will be "/tmp/.yarn-cache-1000".
 10%|█         | 2/20 [00:04<00:36,  2.04s/it]

...

warning Skipping preferred cache folder "/home/jovyan/.cache/yarn" because it is not writable.
warning Selected the next writable cache folder in the list, will be "/tmp/.yarn-cache-1000".
 95%|█████████▌| 19/20 [00:39<00:01,  1.74s/it]

yarn install v1.22.17
[1/4] Resolving packages...
success Already up-to-date.
Done in 0.08s.


warning Skipping preferred cache folder "/home/jovyan/.cache/yarn" because it is not writable.
warning Selected the next writable cache folder in the list, will be "/tmp/.yarn-cache-1000".
100%|██████████| 20/20 [00:41<00:00,  2.09s/it]

Done! 0 / 20  failed, 0 flagged.


{'flagged': [],
 'failed': [],
 'success': ['46981bb8-ed82-4604-ad70-45e7b020c2de',
  '9ec93af0-c2fd-4f43-ab0d-b9b3e39ec1ef',
  '6086f11f-04fe-487d-8cc3-e35c318b17c1',
  '6984738a-d17a-4b44-b786-7bafec87be0f',
  '7e19037c-198b-4dbf-8c4d-acd1b609c4c6',
  '0490bdf9-15fa-4dbf-ae4a-0db4e8ea0e06',
  '17c64870-d3e6-45b6-a90a-c3ef106c91e7',
  '2023f875-5555-470e-8c68-b26b92d905aa',
  '4440b85d-af5a-4d0f-9177-a39a72bf79e1',
  '827e7618-db5d-4cd5-a5df-8d05fce4f574',
  '04296688-95fb-4d26-855d-bac090714789',
  'b7d4c7aa-ceca-4dc7-91bf-6d1e81d62d46',
  '0e96e261-87dd-47f0-b410-15f9edd3bc4f',
  '2a0112e7-a427-44a6-8184-56aece641595',
  '0cefc51b-91b3-461e-9204-1e5a58869ad6',
  '1db9bbc2-c546-4323-b21b-03947bb6f731',
  '5b82bcef-d4d6-433d-8df3-a64fb9ec113f',
  '05822741-9abf-41a3-a6dd-b912cfa62ac5',
  'fe8c4133-c529-48be-a8f4-2ab62daeabc5',
  'a3cd5963-ac93-4f33-932c-30af79b5f39a']}

#### Once we are happy with the layer we will proceed to migrate all +5000 of them, this could take a long while:

In [ ]:
updatedLayers = {}
for k,v in providers.items():
    print(f'{k}: ')
    updatedLayers = createUpdatelayerSchemas(session, v["ids"])

gee: 


  0%|          | 0/1601 [00:00<?, ?it/s]

yarn install v1.22.17
[1/4] Resolving packages...
success Already up-to-date.
Done in 0.09s.


warning Skipping preferred cache folder "/home/jovyan/.cache/yarn" because it is not writable.
warning Selected the next writable cache folder in the list, will be "/tmp/.yarn-cache-1000".
  0%|          | 1/1601 [00:02<1:13:43,  2.76s/it]

yarn install v1.22.17
[1/4] Resolving packages...
success Already up-to-date.
Done in 0.09s.


...

warning Skipping preferred cache folder "/home/jovyan/.cache/yarn" because it is not writable.
warning Selected the next writable cache folder in the list, will be "/tmp/.yarn-cache-1000".
  1%|          | 27/3507 [00:44<1:29:52,  1.55s/it]

yarn install v1.22.17
[1/4] Resolving packages...
success Already up-to-date.
Done in 0.09s.


warning Skipping preferred cache folder "/home/jovyan/.cache/yarn" because it is not writable.
warning Selected the next writable cache folder in the list, will be "/tmp/.yarn-cache-1000".
  1%|          | 28/3507 [00:46<1:32:15,  1.59s/it]

yarn install v1.22.17
[1/4] Resolving packages...
success Already up-to-date.
Done in 0.09s.


warning Skipping preferred cache folder "/home/jovyan/.cache/yarn" because it is not writable.
warning Selected the next writable cache folder in the list, will be "/tmp/.yarn-cache-1000".


### We check here which layers didn't fullfill migration

In [25]:
layerList = getLayerList(session)

In [22]:
layerMigration = {}
for layer in layerList:
    if layer.attributes.layerConfig.get('lmMetadata') is None:
        if layer.attributes.provider in layerMigration.keys():
            layerMigration[layer.attributes.provider]['count'] += 1
            layerMigration[layer.attributes.provider]['ids'].append(layer.id)
        else:
            layerMigration[layer.attributes.provider] = {'count': 1, 'ids':[layer.id]} 
for k,v in layerMigration.items():
    print(f'{k}: {v["count"]}')

cartodb: 424
wms: 10
gee: 112
leaflet: 113
featureservice: 2


In [8]:
orphanedLayers = []
for k,v in layerMigration.items():
    print(f'{k}: ')
    for layerId in v["ids"]:
        try:
            datasetId = session.get(f'v1/layer/{layerId}').json().get('data',{}).get('attributes',{}).get('dataset')
            data = session.get(f'v1/dataset/{datasetId}/layer/{layerId}').json()
        except HTTPError as e:
            if e != None and '404 Client Error' in e.args[0]:
                #print(e)
                orphanedLayers.append(layerId)
                # session.delete(f'v1/layer/{layerId}')
                
len(orphanedLayers)

cartodb: 
wms: 
gee: 
leaflet: 
featureservice: 
None: 


652

We have detected a large ammount of orphan layers (this means that those layers belongs to an already deleted dataset) After discussing the matter with TG he will proceed to delete those.

#### Lets review the rest of the layers:

In [24]:
updatedLayers={}
for k,v in layerMigration.items():
    # we remove first the orphaned layers from the list
    d = list(set(v["ids"]).difference(set(orphanedLayers)))
    
    print(f'{k}: {len(d)} layers')
    
    updatedLayers[k] = createUpdatelayerSchemas(session, d)
        

cartodb: 8 layers


100%|██████████| 8/8 [00:09<00:00,  1.25s/it]


Done! 0 / 2  failed, 6 flagged.
wms: 0 layers


0it [00:00, ?it/s]


Done! 0 / 0  failed, 0 flagged.
gee: 1 layers


...

Done! 0 / 1  failed, 0 flagged.
leaflet: 0 layers


0it [00:00, ?it/s]


Done! 0 / 0  failed, 0 flagged.
featureservice: 0 layers


0it [00:00, ?it/s]

Done! 0 / 0  failed, 0 flagged.


The layers that are failing or are flagged are layers that are not compliyant with the Layer spec. we are going to review why:
* Some of those layers might directly don't have a layerConfig - we will proceed to delete those.
* The layers that do have layerConfig they seem to:
    * Issue with the account not properly setted. we change the code to default to wri-rw (or better default to the dataset carto account.)
    * Layers from carto that does not use the vector spec. they should be type raster. and reevaluated with Alex from Wri
    

### Masks for widgets of map type:  
This requires to check widgets and to update mask (is based on carto so should be easy enough) and update them.
We will only care about the map widget.

In [6]:
widgetList = getWidgetList(session)

In [7]:
widgetList[2]

**Widget**: [Yearly CO2 emissions](https://api.resourcewatch.org/v1/dataset/89a6358e-27eb-4b9c-9f0d-befc4959f914/widget/8c9072da-7c20-4c74-b5ea-b922dbf3d7be)

#### We will focus on map type widgets

In [8]:
widgetMigration = {'count': 0,
                  'ids':[]}

for widget in widgetList:
    if widget.attributes.widgetConfig.get('type') == 'map' and \
        widget.attributes.widgetConfig.get('paramsConfig', {}).get('mask'):
        widgetMigration['count'] += 1
        widgetMigration['ids'].append(widget.id)

widgetMigration

{'count': 7,
 'ids': [UUID('ec16ca2f-566d-4c3c-a8d3-6f3e170a2a8d'),
  UUID('a47c7e2b-ff24-4b02-934d-0107ff26cd2e'),
  UUID('a4f58ef9-f982-4b28-b1c5-6c2013fe0546'),
  UUID('dbd066c0-a1c0-4e0c-9eec-9294b96b49ef'),
  UUID('2d80df96-f93e-4a1b-95fc-4f1241139c0e'),
  UUID('8aed1752-387c-4669-8ffd-064998ae666d'),
  UUID('cc096598-593f-41c1-b660-0ec4aa5c2127')]}

In [9]:
orphanedWidget = []
for widgetId in widgetMigration["ids"]:
    try:
        datasetId = session.get(f'v1/widget/{widgetId}').json().get('data',{}).get('attributes',{}).get('dataset')
        data = session.get(f'v1/dataset/{datasetId}/widget/{widgetId}').json()
    except HTTPError as e:
        if e != None and '404 Client Error' in e.args[0]:
            #print(e)
            orphanedWidget.append(layerId)
            # session.delete(f'v1/layer/{layerId}')
                
len(orphanedWidget)

0

In [20]:
updatedwidgets={}
d = list(set(widgetMigration["ids"]).difference(set(orphanedWidget)))
    
print(f'widgets: {len(d)} widdgets')
    
updatedwidgets = createUpdateMapWidgetSchemas(session, d)

widgets: 7 widdgets


100%|██████████| 7/7 [00:10<00:00,  1.56s/it]

Done! 0 / 7  failed, 0 flagged.
